### Install Libraries

!pip install sklearn
!pip install nltk
!pip install keras
!pip install seaborn
!pip install tensorflow
!pip install biobert-embedding

In [3]:
import io
#from Bio import Entrez, Medline
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import re
import collections
import pandas as pd
import numpy as np
from tqdm import tqdm
#tqdm.pandas(desc="progress-bar")
#from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
#import gensim
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier 
#from xgboost import XGBClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,make_scorer,precision_score,recall_score,roc_auc_score,f1_score
from sklearn.model_selection import GridSearchCV
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
#from gensim.models.doc2vec import TaggedDocument
import re
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import urllib.request, urllib.error, urllib.parse
import json
import os
from pprint import pprint
import nltk
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
#import plotly.graph_objs as go
#import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
#import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
#from plotly.offline import iplot
#cufflinks.go_offline()
#cufflinks.set_config_file(world_readable=True, theme='pearl')
from bs4 import BeautifulSoup
#from rouge import Rouge
from sklearn.decomposition import PCA

import warnings
warnings.simplefilter("ignore", UserWarning)
import getpass
import glob

from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation 

from biobert_embedding.embedding import BiobertEmbedding

2022-09-29 18:22:45.298825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 18:22:46.133981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/oss/hadoop/lib/native/
2022-09-29 18:22:46.134016: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-29 18:22:46.222441: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-29 18:22:48.0

#### Read metadata .csv File

In [43]:
df_covid19_raw=pd.read_csv('./data/metadata.csv')

/opt/oss/conda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (5,13,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [51]:
df_covid19_raw.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139061 entries, 0 to 139060
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   cord_uid           139061 non-null  object
 1   abstract           139061 non-null  object
 2   Abstract_Cat_List  139061 non-null  object
dtypes: object(3)
memory usage: 3.2+ MB


#### Append end: at the end of the file

In [45]:
def appendAbstract(cols):
    '''
    @Author - Fakhare Alam
    '''
    abstract = cols[0]
    if pd.isnull(abstract):
        return 'NA'
    else:
        return abstract+'end:'
    
df_covid19_raw['abstract'] = df_covid19_raw[['abstract']].apply(appendAbstract,axis=1)

#### Define master list for different categories

In [108]:
master_cat_list=['objective:', 'background:', 'background and objectives:', 'context:', 'background and purpose:',
                 'purpose:', 'importance:', 'introduction:', 'aim:', 'rationale:', 'goal:', 'context:', 'hypothesis:'
                ,'population:', 'participant:', 'sample:', 'subject:', 'patient:','patient 2:','patient 1:',
                 'intervention:', 'diagnosis:','prognosis:'
                ,'outcome:', 'measure:', 'variable:', 'assessment:'
                ,'method:','methods:' ,'setting:', 'design:', 'material:', 'procedure:', 'process:', 'methodology:'
                ,'result:','results:','finding:',
                 'conclusion:','conclusions:', 'implication:', 'discussion:', 'interpretation table:','end:']


#### Looping to find keyword in each abstract

In [109]:
def abstractCatList(cols):
    abstract=cols['abstract']
    abstract_cat_list={}
    for cat in master_cat_list:
        indexpos=abstract.lower().find(cat.lower())
        if (indexpos!=-1):
            abstract_cat_list[indexpos]=cat
    return abstract_cat_list

df_covid19_raw['Abstract_Cat_List'] = df_covid19_raw[['abstract']].apply(abstractCatList,axis=1)    

In [110]:
df_covid19_raw=df_covid19_raw[['cord_uid','abstract','Abstract_Cat_List']]

#### Create new dataframe with filled column

In [ ]:
df_covid19_raw_split=pd.DataFrame(columns=df_covid19_raw.columns)

for index, row in df_covid19_raw.iterrows():
    #print(row('Abstract'))
    if(index % 1000==0):
        print('completed',index)
    od = collections.OrderedDict(sorted(row['Abstract_Cat_List'].items()))
    key_list=list(od.keys())
    s=row['abstract'].lower()
    for i in range(0,len(key_list)-1,1):
        try:
            start=row['Abstract_Cat_List'][key_list[i]]
            end=row['Abstract_Cat_List'][key_list[i+1]]
            result = re.search('%s(.*)%s' % (start, end), s).group(1)
            row[start]=result
        except:
            continue
    df_covid19_raw_split=df_covid19_raw_split.append(row,ignore_index=True)


In [112]:
df_covid19_raw_split.fillna('',inplace=True)

In [114]:
A=['objective:', 'background:', 'background and objectives:', 'context:','purpose:', 'importance:', 'introduction:', 'aim:', 'rationale:', 'goal:', 'context:', 'hypothesis:']
df_covid19_raw_split['|a|']=df_covid19_raw_split[A].apply(lambda x: ''.join(x), axis=1)

In [115]:
P=['population:', 'participant:', 'sample:', 'subject:', 'patient:']
df_covid19_raw_split['|p|']=df_covid19_raw_split[P].apply(lambda x: ''.join(x), axis=1)

In [116]:
I=['diagnosis:','intervention:']
df_covid19_raw_split['|i|']=df_covid19_raw_split[I].apply(lambda x: ''.join(x), axis=1)

In [117]:
O=['outcome:', 'measure:', 'variable:', 'assessment:']
df_covid19_raw_split['|o|']=df_covid19_raw_split[O].apply(lambda x: ''.join(x), axis=1)

In [118]:
M=['method:','methods:' ,'setting:', 'design:', 'material:', 'procedure:', 'process:', 'methodology:']
df_covid19_raw_split['|m|']=df_covid19_raw_split[M].apply(lambda x: ''.join(x), axis=1)

In [119]:
R=['result:','results:','finding:']
df_covid19_raw_split['|r|']=df_covid19_raw_split[R].apply(lambda x: ''.join(x), axis=1)


In [120]:
C=['conclusion:','conclusions:', 'implication:', 'discussion:']
df_covid19_raw_split['|c|']=df_covid19_raw_split[C].apply(lambda x: ''.join(x), axis=1)

In [122]:
aimoprc_list=['|a|', '|i|', '|m|', '|o|','|p|', '|r|', '|c|']
# Test
df_covid19_PICO=pd.DataFrame(columns=['sent','aimoprc_category'])
for aimoprc in aimoprc_list:
    df_temp_covid19_PICO=pd.DataFrame(columns=['sent','aimoprc_category'])
    df_temp_covid19_PICO['sent']=df_covid19_raw_split[aimoprc]
    df_temp_covid19_PICO['aimoprc_category']=aimoprc
    df_covid19_PICO=df_covid19_PICO.append(df_temp_covid19_PICO)

In [123]:
df_covid19_PICO.replace('', np.nan, inplace=True)
df_covid19_PICO.dropna(inplace=True)
df_covid19_PICO['aimoprc_category']=df_covid19_PICO['aimoprc_category'].map({'|a|':'A', '|i|':'I', '|m|':'M', '|o|':'O','|p|':'P', '|r|':'R', '|c|':'C'})

In [124]:
rulesCO = {'C':'O'}
# Merge {M,P},{C,O}
df_covid19_PICO['aimoprc_category'].replace(rulesCO,inplace=True)
rulesMP = {'M':'P'}
df_covid19_PICO['aimoprc_category'].replace(rulesMP,inplace=True)

# Drop NULL Sentences
df_covid19_PICO.dropna(axis=0,how='any',inplace=True)

In [137]:
df_covid19_PICO['aimoprc_category'].value_counts()

O    10331
A    10152
P    10127
R     9683
I       52
Name: aimoprc_category, dtype: int64

#### Write to new file

In [126]:
df_covid19_PICO.to_csv('./data/df_covid19_PICO.csv',index=False)
#A->P->I->R-O